In [2]:
import pandas as pd
import numpy as np
#from sklearn.decomposition import PCA,TruncatedSVD,NMF
#from sklearn.preprocessing import Normalizer
import argparse
import time
import pickle as pkl
import glob

In [3]:
def year_binner(year,val=10):
    return year - year%val

In [4]:
def dim_reduction(df,rows):
    df_svd = TruncatedSVD(n_components=300, n_iter=10, random_state=1991)
    print(f'Explained variance ratio {(df_svd.fit(df).explained_variance_ratio_.sum()):2.3f}')
    #df_list=df_svd.fit(df).explained_variance_ratio_
    df_reduced = df_svd.fit_transform(df)
    df_reduced = Normalizer(copy=False).fit_transform(df_reduced)
    df_reduced=pd.DataFrame(df_reduced,index=rows)
    #df_reduced.reset_index(inplace=True)

    df_reduced.index = pd.MultiIndex.from_tuples(df_reduced.index, names=['common', 'time'])
    return df_reduced

all_pkl_files=[]
pkl_file_sizes=[]
path='/data/dharp/compounds/datasets/compounds/'
for filename in glob.glob(path+'*pkl'):
    all_pkl_files.append(filename)
all_pkl_files

novel_compounds=compounds.loc[compounds.groupby(['modifier','head'])['time'].transform('count').eq(1)]
novel_compounds_2000=novel_compounds.loc[novel_compounds.time==2000]
novel_compounds_2000_list=list(zip(novel_compounds_2000['modifier'].tolist(),novel_compounds_2000['head'].tolist()))
display(len(novel_compounds_2000_list))

maske_list=novel_compounds.loc[novel_compounds.time==1990]
maske_list=list(zip(maske_list['modifier'].tolist(),maske_list['head'].tolist()))
display(len(maske_list))

novel_compounds_new=compounds.loc[compounds.groupby(['modifier','head'])['time'].transform('count').eq(2)]
novel_compounds_1990=novel_compounds_new.loc[novel_compounds_new.time.isin([1990,2000])]
novel_compounds_1990_list=list(zip(novel_compounds_1990['modifier'].tolist(),novel_compounds_1990['head'].tolist()))
novel_compounds_1990_list=list(set(novel_compounds_1990_list))
display(len(novel_compounds_1990_list))

In [5]:
    comp_list = [pd.read_pickle(filename) for filename in glob.glob('/data/dharp/compounds/datasets/compounds/*.pkl')]        

    compounds=pd.concat(comp_list)

In [6]:
    print("CompoundCentric Model")
    print("Loading the constituent and compound vector datasets")



    compounds.year=compounds.year.astype("int32")
    compounds=compounds.query('1950 <= year').copy()

    compounds['count']=compounds['count'].astype("int64")
    compounds.modifier=compounds.modifier.replace(r'_(NOUN|PROPN)$', r'', regex=True)

CompoundCentric Model
Loading the constituent and compound vector datasets


KeyboardInterrupt: 

In [7]:
compounds['head']=compounds['head'].replace(r'_(NOUN|PROPN)$', r'', regex=True)  

compounds['common']=compounds['modifier']+" "+compounds['head']
compounds

,modifier,head,context,year,count,common
0,#,#,format_NOUN,1981,1,# #
1,#,#,format_NOUN,1983,1,# #
2,#,#,format_NOUN,1984,3,# #
3,#,#,format_NOUN,1985,4,# #
4,#,#,format_NOUN,1987,2,# #
...,...,...,...,...,...,...
16944983,►,corporation,number_NOUN,2001,6,► corporation
16944984,►,corporation,number_NOUN,2002,4,► corporation
16944985,►,corporation,number_NOUN,2003,6,► corporation
16944986,►,corporation,number_NOUN,2004,6,► corporation


In [8]:
compounds['time']=year_binner(compounds['year'].values,10)

In [11]:
comp_counts=compounds.groupby(['common','time'])['count'].sum()
comp_counts

common  time
# #     1950      1
        1960      2
        1970      2
        1980    406
        1990    474
               ... 
      2000    298
      1990      3
        2000     72
      1990     36
        2000    510
Name: count, Length: 6488044, dtype: int64

In [13]:
comp_counts.to_csv("/data/dharp/compounds/datasets/comp_counts.csv",sep="\t",header=False)

In [9]:
    compounds['time']=year_binner(compounds['year'].values,10)
    compounds=compounds.loc[compounds.groupby(['common','time'])['count'].transform('sum').gt(200)]
    modifiers_list=compounds.modifier.unique()
    heads_list=compounds['head'].unique()
    compounds=compounds.groupby(['common','context','time'])['count'].sum()
    compounds

common  context      time
# #     exact_ADJ    1980      1
                     1990     58
        follow_VERB  1980      2
                     1990     14
        format_NOUN  1980     14
                            ... 
      s_VERB       2000    473
      -_ADJ        2000    149
        mid_NOUN     2000    149
      -_ADJ        2000    255
        mid_NOUN     2000    255
Name: count, Length: 28731672, dtype: int64

In [10]:
modifiers_list[:10]

array(['#', '%', "'", '+5', '+', '-0.50.6', '-50,000', '-80', '-', '11th'],
      dtype=object)

In [12]:
    mod_list = [pd.read_pickle(filename) for filename in glob.glob('/data/dharp/compounds/datasets/modifiers/*.pkl')]        

    modifiers=pd.concat(mod_list)

In [13]:
    print(modifiers.shape)
    modifiers.year=modifiers.year.astype("int32")
    modifiers.modifier=modifiers.modifier.replace(r'_(NOUN|PROPN)$', r'', regex=True)
    modifiers=modifiers.query('1950 <= year').copy()
    modifiers=modifiers.loc[modifiers.modifier.isin(modifiers_list)]
    modifiers['count']=modifiers['count'].astype("float64")
    modifiers.columns=['common','context','year','count']
    print(modifiers.shape)
    modifiers['time']=year_binner(modifiers['year'].values,10)
    modifiers

(553725078, 4)
(372005317, 4)


,common,context,year,count,time
0,#,#_NOUN,1981,1.0,1980
1,#,#_NOUN,1983,1.0,1980
2,#,#_NOUN,1984,3.0,1980
3,#,#_NOUN,1985,4.0,1980
4,#,#_NOUN,1987,2.0,1980
...,...,...,...,...,...
13146983,◦,c_NOUN,2004,10.0,2000
13146984,◦,c_NOUN,2005,6.0,2000
13146985,◦,c_NOUN,2006,8.0,2000
13146986,◦,c_NOUN,2007,10.0,2000


In [14]:
    modifiers['common']=modifiers['common']+"_m"
    modifiers=modifiers.groupby(['common','context','time'])['count'].sum()
    modifiers

common  context   time
#_m     #_NOUN    1950      5.0
                  1960      3.0
                  1970     13.0
                  1980    293.0
                  1990    457.0
                          ...  
_m     mid_NOUN  2000    404.0
        s_NOUN    1990      3.0
                  2000     72.0
        _NOUN    1990      6.0
                  2000    187.0
Name: count, Length: 20939595, dtype: float64

In [15]:
    head_list = [pd.read_pickle(filename) for filename in glob.glob('/data/dharp/compounds/datasets/heads/*.pkl')]        

    heads=pd.concat(head_list)

In [16]:
    print(heads.shape)
    heads.year=heads.year.astype("int32")
    heads['head']=heads['head'].replace(r'_(NOUN|PROPN)$', r'', regex=True)
    heads=heads.query('1950 <= year').copy()
    heads=heads.loc[heads['head'].isin(heads_list)]
    heads['count']=heads['count'].astype("float64")
    heads.columns=['common','context','year','count']
    print(heads.shape)
    heads['time']=year_binner(heads['year'].values,10)
    heads

(529645364, 4)
(355570840, 4)


,common,context,year,count,time
0,#,#_NOUN,1981,1.0,1980
1,#,#_NOUN,1983,1.0,1980
2,#,#_NOUN,1984,3.0,1980
3,#,#_NOUN,1985,4.0,1980
4,#,#_NOUN,1987,2.0,1980
...,...,...,...,...,...
15819342,➪,choose_VERB,2004,12.0,2000
15819343,➪,choose_VERB,2005,26.0,2000
15819344,➪,choose_VERB,2006,8.0,2000
15819345,➪,choose_VERB,2007,16.0,2000


In [17]:
    heads['common']=heads['common']+"_h"
    heads=heads.groupby(['common','context','time'])['count'].sum()
    heads

common  context   time
#_h     #_NOUN    1950      19.0
                  1960      71.0
                  1970     123.0
                  1980     611.0
                  1990    1011.0
                           ...  
_h     late_ADJ  1990      18.0
                  2000     520.0
        s_VERB    1980       2.0
                  1990      19.0
                  2000     473.0
Name: count, Length: 18992533, dtype: float64

In [18]:
df=pd.concat([heads,modifiers,compounds], sort=True)

In [19]:
type(df)

pandas.core.series.Series

In [20]:
dtype = pd.SparseDtype(float, fill_value=0)
df=df.astype(dtype)

In [21]:
type(df)

pandas.core.series.Series

In [22]:
df, rows, _ = df.sparse.to_coo(row_levels=['common','time'],column_levels=['context'],sort_labels=False)

In [23]:
print('Running SVD')   
df_reduced=dim_reduction(df,rows)

Running SVD
Explained variance ratio 0.908


In [24]:
compounds_reduced=df_reduced.loc[df_reduced.index.get_level_values(0).str.contains(r'\w \w')]
compounds_reduced.reset_index(inplace=True)
    #print(compounds_reduced.head())
compounds_reduced[['modifier','head']]=compounds_reduced['common'].str.split(' ', n=1,expand=True).copy()
compounds_reduced

/data/dharp/packages/miniconda3/envs/spacied/lib/python3.9/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,common,time,0,1,2,3,4,5,6,7,...,292,293,294,295,296,297,298,299,modifier,head
0,'s quilt,1990,0.007832,0.059829,-0.031936,-0.066304,0.007579,-0.042816,-0.015612,-0.002561,...,-0.005688,-0.025025,-0.016804,-0.016301,0.007180,0.046095,0.050832,-0.006269,'s,quilt
1,+1 v,1990,0.010044,0.095959,-0.005823,-0.056895,0.016886,-0.085580,-0.028374,-0.035984,...,0.023574,0.038533,0.016283,0.072119,-0.070514,-0.032994,0.006339,-0.031224,+1,v
2,+5 v,1980,0.021831,0.225970,0.050935,0.010884,-0.021162,0.068693,0.115418,0.262376,...,0.047127,0.010924,-0.036636,0.041017,-0.031380,-0.041131,-0.050458,-0.026649,+5,v
3,+5 v,1990,0.020505,0.206646,0.038952,-0.000764,-0.019165,0.060420,0.106927,0.232739,...,0.041276,0.020125,-0.019265,0.053320,-0.016862,-0.055837,-0.079821,-0.029404,+5,v
4,+5 v,2000,0.024218,0.258608,0.067927,0.025135,-0.024364,0.081789,0.129163,0.302154,...,0.055252,0.006387,-0.054912,0.033273,-0.044528,-0.028714,-0.018798,-0.027256,+5,v
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861937,zymogen granule,2000,0.013367,0.092165,-0.042929,-0.091425,0.005232,-0.029679,0.000581,0.015767,...,-0.088507,0.002488,0.072503,0.070204,-0.011515,0.063538,-0.066650,0.008026,zymogen,granule
861938,¼ inch,2000,0.005826,0.045735,0.001790,-0.015764,0.005044,-0.025822,-0.000190,-0.003737,...,-0.003438,0.053666,0.038193,0.071045,-0.063428,-0.034716,0.015961,-0.026191,¼,inch
861939,¼ teaspoon,2000,0.004635,0.042301,-0.008956,-0.032373,0.001632,-0.017565,0.001033,0.002470,...,-0.032731,0.127252,-0.046452,0.026440,0.059794,0.021990,0.001104,-0.143370,¼,teaspoon
861940,½ cup,2000,0.005795,0.037588,-0.027624,-0.021103,0.004270,-0.017355,-0.005591,0.000652,...,0.024897,0.015057,0.138693,-0.005621,0.056439,-0.003240,0.033509,0.018779,½,cup


In [25]:
    heads_reduced=df_reduced.loc[df_reduced.index.get_level_values(0).str.endswith(r'_h')]
    heads_reduced.reset_index(inplace=True)
    heads_reduced['head']=heads_reduced['common'].str.replace(r'_h$', r'', regex=True)
    heads_reduced.drop(['common'],axis=1,inplace=True)

    modifiers_reduced=df_reduced.loc[df_reduced.index.get_level_values(0).str.endswith(r'_m')]
    modifiers_reduced.reset_index(inplace=True)   
    modifiers_reduced['modifier']=modifiers_reduced['common'].str.replace(r'_m$', r'', regex=True)
    modifiers_reduced.drop(['common'],axis=1,inplace=True)

/tmp/ipykernel_8723/1365444598.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  heads_reduced['head']=heads_reduced['common'].str.replace(r'_h$', r'', regex=True)
/data/dharp/packages/miniconda3/envs/spacied/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/tmp/ipykernel_8723/1365444598.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [26]:
modifiers_reduced

,time,0,1,2,3,4,5,6,7,8,...,291,292,293,294,295,296,297,298,299,modifier
0,1950,0.007835,0.060462,-0.015356,-0.025647,-0.005441,0.015143,0.025845,0.062815,0.002941,...,-0.006304,-0.008579,-0.028010,0.006838,-0.033940,0.014700,0.009902,-0.018421,0.003210,#
1,1960,0.007134,0.056907,-0.013401,-0.022211,-0.005303,0.015603,0.026494,0.062857,0.003438,...,-0.011934,-0.003448,-0.026292,0.010916,-0.036334,0.018390,0.007375,-0.014578,0.002691,#
2,1970,0.006889,0.056524,-0.013239,-0.021825,-0.005372,0.016017,0.026520,0.062745,0.003507,...,-0.013493,-0.002030,-0.026412,0.012174,-0.037289,0.019302,0.006600,-0.013645,0.002481,#
3,1980,0.026035,0.129040,-0.058332,-0.104494,0.003879,-0.030774,-0.004728,0.028367,0.009893,...,-0.035324,0.045967,0.133936,-0.016837,-0.055994,0.062812,0.160412,-0.135332,0.141433,#
4,1990,0.023219,0.118310,-0.077671,-0.108093,-0.002848,0.000169,-0.016769,0.004630,0.002887,...,0.018449,-0.034951,0.078649,0.086859,0.136818,0.097522,0.109699,-0.128057,0.153635,#
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194292,1990,0.004317,0.021240,-0.013028,-0.027820,0.015827,-0.008836,-0.005015,0.004277,0.007135,...,-0.023574,-0.022039,0.000890,-0.024588,-0.058914,0.006930,0.014460,-0.032223,0.008860,
194293,2000,0.004429,0.022302,-0.013602,-0.028716,0.015099,-0.007824,-0.004278,0.005767,0.007426,...,-0.020485,-0.020948,-0.007028,-0.014010,-0.046457,0.004226,0.009519,-0.025171,0.013442,
194294,1980,0.006344,0.017000,-0.012368,-0.024793,0.012108,-0.034678,-0.024798,-0.026300,0.679882,...,-0.008664,-0.018224,0.016162,-0.016402,-0.009420,-0.029211,0.003552,0.006775,-0.011508,
194295,1990,0.006474,0.017976,-0.012961,-0.026077,0.012194,-0.035301,-0.024934,-0.026235,0.679285,...,-0.011038,-0.019526,0.019600,-0.016150,-0.013668,-0.026428,0.004046,0.007592,-0.010746,


In [27]:
        compounds_reduced.set_index(['modifier','head','time'],inplace=True)
        heads_reduced.set_index(['head','time'],inplace=True)
        modifiers_reduced.set_index(['modifier','time'],inplace=True)

In [28]:
compounds_reduced

common         0         1         2  \
modifier head     time                                                  
's       quilt    1990         's quilt  0.007832  0.059829 -0.031936   
+1       v        1990             +1 v  0.010044  0.095959 -0.005823   
+5       v        1980             +5 v  0.021831  0.225970  0.050935   
                  1990             +5 v  0.020505  0.206646  0.038952   
                  2000             +5 v  0.024218  0.258608  0.067927   
...                                 ...       ...       ...       ...   
zymogen  granule  2000  zymogen granule  0.013367  0.092165 -0.042929   
¼        inch     2000           ¼ inch  0.005826  0.045735  0.001790   
         teaspoon 2000       ¼ teaspoon  0.004635  0.042301 -0.008956   
½        cup      2000            ½ cup  0.005795  0.037588 -0.027624   
         stick    2000          ½ stick  0.000177  0.001688 -0.000688   

                               3         4         5         6         7  \
modifier head     time                                                     
's       quilt    1990 -0.066304  0.007579 -0.042816 -0.015612 -0.002561   
+1       v        1990 -0.056895  0.016886 -0.085580 -0.028374 -0.035984   
+5       v        1980  0.010884 -0.021162  0.068693  0.115418  0.262376   
                  1990 -0.000764 -0.019165  0.060420  0.106927  0.232739   
                  2000  0.025135 -0.024364  0.081789  0.129163  0.302154   
...                          ...       ...       ...       ...       ...   
zymogen  granule  2000 -0.091425  0.005232 -0.029679  0.000581  0.015767   
¼        inch     2000 -0.015764  0.005044 -0.025822 -0.000190 -0.003737   
         teaspoon 2000 -0.032373  0.001632 -0.017565  0.001033  0.002470   
½        cup      2000 -0.021103  0.004270 -0.017355 -0.005591  0.000652   
         stick    2000 -0.001408  0.000020 -0.000318  0.004857  0.002774   

                               8  ...       290       291       292       293  \
modifier head     time            ...                                           
's       quilt    1990  0.000048  ... -0.000179  0.029610 -0.005688 -0.025025   
+1       v        1990  0.007984  ... -0.032750  0.036233  0.023574  0.038533   
+5       v        1980  0.016257  ...  0.024447  0.031855  0.047127  0.010924   
                  1990  0.012000  ...  0.019837  0.026725  0.041276  0.020125   
                  2000  0.020684  ...  0.034007  0.035541  0.055252  0.006387   
...                          ...  ...       ...       ...       ...       ...   
zymogen  granule  2000  0.011679  ...  0.027478  0.020900 -0.088507  0.002488   
¼        inch     2000  0.001694  ... -0.018226  0.071838 -0.003438  0.053666   
         teaspoon 2000 -0.002615  ... -0.106843  0.005386 -0.032731  0.127252   
½        cup      2000  0.002227  ... -0.030345  0.057428  0.024897  0.015057   
         stick    2000  0.000653  ... -0.213508  0.095726 -0.046268  0.222875   

                             294       295       296       297       298  \
modifier head     time                                                     
's       quilt    1990 -0.016804 -0.016301  0.007180  0.046095  0.050832   
+1       v        1990  0.016283  0.072119 -0.070514 -0.032994  0.006339   
+5       v        1980 -0.036636  0.041017 -0.031380 -0.041131 -0.050458   
                  1990 -0.019265  0.053320 -0.016862 -0.055837 -0.079821   
                  2000 -0.054912  0.033273 -0.044528 -0.028714 -0.018798   
...                          ...       ...       ...       ...       ...   
zymogen  granule  2000  0.072503  0.070204 -0.011515  0.063538 -0.066650   
¼        inch     2000  0.038193  0.071045 -0.063428 -0.034716  0.015961   
         teaspoon 2000 -0.046452  0.026440  0.059794  0.021990  0.001104   
½        cup      2000  0.138693 -0.005621  0.056439 -0.003240  0.033509   
         stick    2000  0.231286  0.107461  0.227733  0.074520  0.204416   

                             299  
modifier head   

In [29]:
compounds_reduced.drop('common',axis=1).to_csv("/data/dharp/compounds/datasets/google_big/compounds_reduced.csv",sep="\t",header=False)

In [30]:
modifiers_reduced.to_csv("/data/dharp/compounds/datasets/google_big/modifiers_reduced.csv",sep="\t",header=False)
heads_reduced.to_csv("/data/dharp/compounds/datasets/google_big/heads_reduced.csv",sep="\t",header=False)

In [28]:
with open('/data/dharp/compounds/datasets/modifier_list.txt','w') as f:
    for mod in modifiers_list:
        f.write(f'{mod}\n')

In [29]:
with open('/data/dharp/compounds/datasets/head_list.txt','w') as f:
    for head in heads_list:
        f.write(f'{head}\n')